In [ ]:
# torchserve --start --model-store model_store --models model_store/STTS2Encoder.mar,model_store/STTS2Catalan.mar --disable-token-auth  --enable-model-api --foreground --no-config-snapshots
# torch-model-archiver -f --model-name STTS2Catalan --version 1.0 --serialized-file NewStyleTTS2Catalan.pth --model-file models.py --export-path model_store --handler handler.py --extra-files config.json
# torchserve --start --model-store model_store --models model_store/STTS2Encoder.mar,model_store/STTS2Catalan.mar --workflow-store wf_store/ --disable-token-auth  --enable-model-api --foreground --no-config-snapshots

In [ ]:
audio_file_path = "reference_audio.mp3"

## Testing the Catalan Style Encoder Model by passing Audio file as input

In [ ]:
# Catalan Encoder
import requests
import os

# Define the URL
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2Encoder"

# Define the file path (assuming the script is running in the same directory as the audio file)
file_path = os.path.join(os.getcwd(), audio_file_path)

# Open the file in binary mode and send as part of the request
with open(file_path, 'rb') as audio_file:
    files = {'audio': audio_file}
    response = requests.post(url, files=files)

# Print the response (optional, for debugging)
print(response.status_code)
print(response)
print(response.json())
ref_s = response.json()['ref_s']
hash_s = response.json()['hash']

## Testing Catalan TTS model

In [ ]:
# Catalan TTS API with ref_s
import requests
from IPython.display import Audio
import IPython.display as ipd
# Define the URL
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2Catalan"

# Define the JSON data
data = {
    "text": "Quan vaig tornar al poble, aquell estiu, i vaig veure els carrers plens de gent, de música i d’alegria, em vaig sentir com si la vida m’hagués regalat una postal dels meus millors somnis. Aquell somriure que se m’escapava no era només meu; era el batec col·lectiu d’una felicitat compartida, d’instants que mai volen desaparèixer.",
    "ref_s": ref_s
}

# Send the POST request with JSON data
response = requests.post(url, json=data)
# Preview the audio file
ipd.display(Audio(response.content, autoplay=True))

## Testing Catalan TTS model with hash(Catalan Style Encoder Hash) as input

In [ ]:
# Catalan TTS API with hash
import requests
from IPython.display import Audio
import IPython.display as ipd
# Define the URL
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2Catalan"


data = {
    "text": "hóla sóy maría y téngo ún jamón y háblo cón múchás érrés y múchás éssés",
    "hash": hash_s
}

response = requests.post(url, json=data)
# Preview the audio file
Audio(response.content, autoplay=True)

## Testing the Spanish Style Encoder Model by passing Audio file as input

In [ ]:
# Spanish Encoder
import requests
import os

# Define the URL
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2SpanishEncoder"

# Define the file path (assuming the script is running in the same directory as the audio file)
file_path = os.path.join(os.getcwd(), audio_file_path)

# Open the file in binary mode and send as part of the request
with open(file_path, 'rb') as audio_file:
    files = {'audio': audio_file}
    response = requests.post(url, files=files)

# Print the response (optional, for debugging)
print(response.status_code)
print(response)
print(response.json())
ref_s = response.json()['ref_s']
hash_s = response.json()['hash']

## Testing Spanish TTS model

In [ ]:
# Spanish TTS API with ref_s
import requests
import base64
from IPython.display import Audio
import IPython.display as ipd
# Define the URL
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2Spanish"

# Define the JSON data
data = {
    "text": "En un pequeño pueblo rodeado de montañas y ríos cristalinos, vivía una joven llamada Elena. Desde pequeña, había sentido una profunda fascinación por las estrellas y los misterios del universo. Todas las noches, subía a la colina más alta y, con su viejo telescopio, observaba el cielo estrellado, imaginando historias sobre los mundos lejanos que tal vez nunca podría visitar. Una noche, mientras contemplaba la luna llena, escuchó un sonido extraño proveniente del bosque. Sin pensarlo dos veces, decidió investigar. Caminó entre los árboles altos y frondosos, siguiendo el eco de una melodía lejana. Finalmente, llegó a un claro iluminado por una luz tenue y misteriosa. En el centro, había un anciano de cabello plateado tocando una flauta de madera. Su música era hipnotizante, como si cada nota contara una historia secreta del universo. El anciano la miró y sonrió. 'Elena', dijo con voz serena, 'las estrellas no solo están en el cielo; también viven en los sueños de quienes se atreven a imaginar'. En ese momento, Elena comprendió que su destino no estaba limitado por la distancia o la realidad, sino por los límites de su propia imaginación.",
}

# Send the POST request with JSON data
response = requests.post(url, json=data)
speech = base64.b64decode(response.content)
# Preview the audio file
ipd.display(Audio(response, autoplay=True))


## Stress Testing

In [ ]:
import requests
import json
import IPython.display as ipd
import concurrent.futures
import time

# Define the URL
url = "http://127.0.0.1:8080/predictions/STTS2Catalan"
url = "https://2lfnp43olqz2nv-8080.proxy.runpod.net/predictions/STTS2Catalan"


# Define the JSON data
data = {
    "text": "Quan vaig tornar al poble, aquell estiu, i vaig veure els carrers plens de gent, de música i d’alegria, em vaig sentir com si la vida m’hagués regalat una postal dels meus millors somnis. Aquell somriure que se m’escapava no era només meu; era el batec col·lectiu d’una felicitat compartida, d’instants que mai volen desaparèixer.",
}
i=0
start_time = time.time()

def send_request(data):
    global i
    response = requests.post(url, json=data)
    ipd.display(ipd.Audio(response.content, autoplay=True))
    i+=1
    print(f"Completed {i} requests in {time.time() - start_time} seconds")
    return response

# Send 10 concurrent POST requests with JSON data
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(send_request, data) for _ in range(100)]
    responses = [future.result() for future in concurrent.futures.as_completed(futures)]

print(f"Time Taken: {time.time() - start_time}")
print(f"Average Time Taken: {(time.time() - start_time) / 100}")
